In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint

In [18]:
training_data = pd.read_csv('training_v6.csv')
# training_data = training_data.drop(['item_cnt_prev_3_mnth_mean'],axis=1)
training_data

,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnthly_mean,item_cnt_prev_3_mnth_mean,item_cnt_prev_4_mnth_mean,avg_item_price_3mnth_ago,item_cnt_mnth
0,0,0,2013,1,32,40,8.000000,0.000000,0.00,0.000000,6.0
1,0,0,2013,1,33,37,3.000000,0.000000,0.00,0.000000,3.0
2,0,0,2013,1,35,40,7.500000,0.000000,0.00,0.000000,1.0
3,0,0,2013,1,43,40,1.000000,0.000000,0.00,0.000000,1.0
4,0,0,2013,1,51,57,2.500000,0.000000,0.00,0.000000,2.0
...,...,...,...,...,...,...,...,...,...,...,...
1609119,59,33,2015,10,22087,83,6.424242,3.333333,2.75,119.000000,6.0
1609120,59,33,2015,10,22088,83,6.187500,5.000000,4.75,119.000000,2.0
1609121,59,33,2015,10,22091,83,2.294118,1.000000,0.75,59.666667,1.0
1609122,59,33,2015,10,22100,42,1.000000,0.333333,0.25,209.666667,1.0


In [19]:
X = training_data.iloc[:,0:10].values
y = training_data.iloc[:,10].values
y = y.clip(0.,)
y = np.log1p(y)
feature_list = [c for c in training_data.columns if c not in 'item_cnt_month']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [25]:
rand_forest = RandomForestRegressor(bootstrap=True,n_estimators=100, n_jobs=-1, random_state=42)
rand_forest.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [26]:
importances = list(rand_forest.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, 
                       importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: item_cnt_prev_3_mnth_mean Importance: 0.55
Variable: item_category_id     Importance: 0.1
Variable: shop_id              Importance: 0.06
Variable: avg_item_price_3mnth_ago Importance: 0.06
Variable: item_cnt_mnth        Importance: 0.06
Variable: item_cnt_mnthly_mean Importance: 0.05
Variable: item_cnt_prev_4_mnth_mean Importance: 0.05
Variable: date_block_num       Importance: 0.04
Variable: item_id              Importance: 0.03
Variable: year                 Importance: 0.01


In [27]:
y_pred = rand_forest.predict(X_test)

In [28]:
RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print("RMSE:", RMSE)

RMSE: 0.2895342747759866
